# Registry

> Unified plugin registry for managing multiple domain-specific plugin systems with configuration persistence

In [ ]:
#| default_exp core.registry

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from pathlib import Path
from typing import Dict, List, Optional, Any, TypeVar, Type

from cjm_fasthtml_plugins.core.metadata import PluginMetadata
from cjm_fasthtml_plugins.core.execution_mode import PluginExecutionMode

## Unified Plugin Registry

The `UnifiedPluginRegistry` manages multiple domain-specific plugin systems (transcription, LLM, image generation, etc.) in a single application. It provides:

- String-based categories (no hardcoded enums)
- Configuration persistence (JSON files)
- Integration with FastHTML settings and resource management
- Support for local and cloud plugins

This registry works with any plugin system built on `cjm-plugin-system`.

In [ ]:
#| export
T = TypeVar('T')

class UnifiedPluginRegistry:
    """Unified registry for multiple domain-specific plugin systems.
    
    Manages plugin managers from different domains (transcription, LLM, etc.)
    and provides a single interface for plugin discovery, configuration,
    and resource management.
    
    Example:
        ```python
        from cjm_plugin_system.core.manager import PluginManager
        from cjm_transcription_plugin_system.plugin_interface import TranscriptionPlugin
        
        # Create registry
        registry = UnifiedPluginRegistry()
        
        # Register transcription plugins
        transcription_mgr = PluginManager(plugin_interface=TranscriptionPlugin)
        registry.register_plugin_manager(
            category="transcription",
            manager=transcription_mgr,
            display_name="Transcription"
        )
        
        # Get all plugins
        all_plugins = registry.get_all_plugins()
        
        # Get plugins by category
        transcription_plugins = registry.get_plugins_by_category("transcription")
        
        # Get manager for specific operations
        mgr = registry.get_manager("transcription")
        ```
    """
    
    def __init__(self, config_dir: Optional[Path] = None):
        """Initialize the unified plugin registry.
        
        Args:
            config_dir: Directory for plugin configuration files (default: 'configs')
        """
        self._managers: Dict[str, Any] = {}  # category -> manager
        self._categories: Dict[str, str] = {}  # category -> display_name
        self._plugins: Dict[str, PluginMetadata] = {}  # unique_id -> metadata
        self._config_dir = config_dir or Path("configs")
        self._config_dir.mkdir(exist_ok=True, parents=True)
    
    def register_plugin_manager(
        self,
        category: str,  # Category name (e.g., "transcription", "llm")
        manager: Any,  # Domain-specific plugin manager
        display_name: Optional[str] = None,  # Display name for UI
        auto_discover: bool = True  # Automatically discover plugins?
    ) -> List[PluginMetadata]:  # List of discovered plugin metadata
        """Register a domain-specific plugin manager.
        
        Args:
            category: String category (e.g., 'transcription')
            manager: The domain-specific plugin manager instance
            display_name: Optional display name for UI
            auto_discover: Automatically discover and register plugins
        
        Returns:
            List of discovered plugin metadata
        """
        self._managers[category] = manager
        self._categories[category] = display_name or category.title()
        
        if auto_discover:
            return self._discover_and_register_plugins(category, manager)
        return []
    
    def _discover_and_register_plugins(
        self,
        category: str,
        manager: Any
    ) -> List[PluginMetadata]:
        """Discover plugins from manager and register their metadata."""
        discovered = manager.discover_plugins()
        plugin_metadatas = []
        
        for plugin_data in discovered:
            # Get config schema from manager
            config_schema = manager.get_plugin_config_schema(plugin_data.name)
            
            # Create plugin metadata
            metadata = PluginMetadata(
                name=plugin_data.name,
                category=category,
                title=config_schema.get('title', plugin_data.name),
                config_schema=config_schema,
                version=plugin_data.version,
                description=config_schema.get('description')
            )
            
            # Check if plugin is configured
            config_file = self._config_dir / f"{metadata.get_unique_id()}.json"
            metadata.is_configured = config_file.exists()
            
            # Store metadata
            self._plugins[metadata.get_unique_id()] = metadata
            plugin_metadatas.append(metadata)
        
        return plugin_metadatas
    
    def get_manager(
        self,
        category: str,  # Category name
        manager_type: Optional[Type[T]] = None  # Optional type hint
    ) -> Optional[T]:  # Plugin manager instance
        """Get plugin manager for a specific category.
        
        Args:
            category: Category name (e.g., 'transcription')
            manager_type: Optional type hint for IDE autocomplete
        
        Returns:
            Plugin manager instance if found, None otherwise
        """
        return self._managers.get(category)
    
    def get_categories(self) -> List[str]:
        """Get all registered categories.
        
        Returns:
            Sorted list of category names
        """
        return sorted(self._categories.keys())
    
    def get_category_display_name(self, category: str) -> str:
        """Get display name for a category.
        
        Args:
            category: Category name
        
        Returns:
            Display name or category name if not set
        """
        return self._categories.get(category, category.title())
    
    def get_plugin(self, unique_id: str) -> Optional[PluginMetadata]:
        """Get plugin metadata by unique ID.
        
        Args:
            unique_id: Plugin unique identifier (format: 'category_name')
        
        Returns:
            Plugin metadata if found, None otherwise
        """
        return self._plugins.get(unique_id)
    
    def get_plugins_by_category(self, category: str) -> List[PluginMetadata]:
        """Get all plugins in a category.
        
        Args:
            category: Category name
        
        Returns:
            List of plugin metadata for the category
        """
        return [p for p in self._plugins.values() if p.category == category]
    
    def get_all_plugins(self) -> List[PluginMetadata]:
        """Get all plugins across all categories.
        
        Returns:
            List of all plugin metadata
        """
        return list(self._plugins.values())
    
    def get_categories_with_plugins(self) -> List[str]:
        """Get categories that have registered plugins.
        
        Returns:
            Sorted list of categories with plugins
        """
        categories = set(p.category for p in self._plugins.values())
        return sorted(categories)
    
    def load_plugin_config(self, unique_id: str) -> Dict[str, Any]:
        """Load saved configuration for a plugin.
        
        Args:
            unique_id: Plugin unique identifier
        
        Returns:
            Configuration dictionary (empty if no config exists)
        """
        config_file = self._config_dir / f"{unique_id}.json"
        if config_file.exists():
            with open(config_file, 'r') as f:
                return json.load(f)
        return {}
    
    def save_plugin_config(self, unique_id: str, config: Dict[str, Any]) -> bool:
        """Save configuration for a plugin.
        
        Args:
            unique_id: Plugin unique identifier
            config: Configuration dictionary to save
        
        Returns:
            True if save succeeded, False otherwise
        """
        try:
            config_file = self._config_dir / f"{unique_id}.json"
            with open(config_file, 'w') as f:
                json.dump(config, f, indent=2)
            
            # Update plugin's configured status
            if unique_id in self._plugins:
                self._plugins[unique_id].is_configured = True
            
            return True
        except Exception as e:
            print(f"Error saving config for {unique_id}: {e}")
            return False
    
    def delete_plugin_config(self, unique_id: str) -> bool:
        """Delete saved configuration for a plugin.
        
        Args:
            unique_id: Plugin unique identifier
        
        Returns:
            True if deletion succeeded, False otherwise
        """
        try:
            config_file = self._config_dir / f"{unique_id}.json"
            if config_file.exists():
                config_file.unlink()
            
            # Update plugin's configured status
            if unique_id in self._plugins:
                self._plugins[unique_id].is_configured = False
            
            return True
        except Exception as e:
            print(f"Error deleting config for {unique_id}: {e}")
            return False

## Example Usage

In [ ]:
# Create a mock plugin manager for testing
from dataclasses import dataclass

@dataclass
class MockPluginData:
    name: str
    version: str

class MockPluginManager:
    """Mock plugin manager for testing."""
    
    def discover_plugins(self):
        return [
            MockPluginData("whisper_tiny", "1.0.0"),
            MockPluginData("whisper_base", "1.0.0")
        ]
    
    def get_plugin_config_schema(self, name: str):
        return {
            "type": "object",
            "title": f"{name.replace('_', ' ').title()} Configuration",
            "description": f"Configuration for {name}",
            "properties": {
                "device": {
                    "type": "string",
                    "enum": ["cpu", "cuda"],
                    "default": "cpu"
                }
            }
        }

In [ ]:
# Example: Create registry and register a plugin manager
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    
    # Register transcription plugin manager
    mock_manager = MockPluginManager()
    discovered = registry.register_plugin_manager(
        category="transcription",
        manager=mock_manager,
        display_name="Transcription Plugins"
    )
    
    print(f"Discovered {len(discovered)} plugins")
    for plugin in discovered:
        print(f"  - {plugin.title} (v{plugin.version})")

Discovered 2 plugins
  - Whisper Tiny Configuration (v1.0.0)
  - Whisper Base Configuration (v1.0.0)


In [ ]:
# Example: Query registry
with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    registry.register_plugin_manager("transcription", MockPluginManager())
    
    # Get categories
    print(f"Categories: {registry.get_categories()}")
    print(f"Categories with plugins: {registry.get_categories_with_plugins()}")
    
    # Get plugins
    plugins = registry.get_plugins_by_category("transcription")
    print(f"\nTranscription plugins: {len(plugins)}")
    for plugin in plugins:
        print(f"  - {plugin.name} ({plugin.get_unique_id()})")
    
    # Get specific plugin
    plugin = registry.get_plugin("transcription_whisper_tiny")
    print(f"\nPlugin metadata:")
    print(f"  Title: {plugin.title}")
    print(f"  Category: {plugin.category}")
    print(f"  Is configured: {plugin.is_configured}")

Categories: ['transcription']
Categories with plugins: ['transcription']

Transcription plugins: 2
  - whisper_tiny (transcription_whisper_tiny)
  - whisper_base (transcription_whisper_base)

Plugin metadata:
  Title: Whisper Tiny Configuration
  Category: transcription
  Is configured: False


In [ ]:
# Example: Save and load plugin configuration
with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    registry.register_plugin_manager("transcription", MockPluginManager())
    
    unique_id = "transcription_whisper_tiny"
    
    # Check initial state
    plugin = registry.get_plugin(unique_id)
    print(f"Initially configured: {plugin.is_configured}")
    
    # Save configuration
    config = {"device": "cuda", "model_size": "tiny"}
    success = registry.save_plugin_config(unique_id, config)
    print(f"Save succeeded: {success}")
    
    # Check updated state
    plugin = registry.get_plugin(unique_id)
    print(f"Now configured: {plugin.is_configured}")
    
    # Load configuration
    loaded_config = registry.load_plugin_config(unique_id)
    print(f"Loaded config: {loaded_config}")
    
    # Delete configuration
    success = registry.delete_plugin_config(unique_id)
    print(f"Delete succeeded: {success}")
    
    # Check final state
    plugin = registry.get_plugin(unique_id)
    print(f"Finally configured: {plugin.is_configured}")

Initially configured: False
Save succeeded: True
Now configured: True
Loaded config: {'device': 'cuda', 'model_size': 'tiny'}
Delete succeeded: True
Finally configured: False


In [ ]:
# Example: Register multiple plugin managers (multi-domain)
class MockLLMManager:
    def discover_plugins(self):
        return [MockPluginData("llama3", "1.0.0"), MockPluginData("gpt4", "1.0.0")]
    def get_plugin_config_schema(self, name: str):
        return {"type": "object", "title": f"{name} Config", "properties": {}}

with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    
    # Register multiple domains
    registry.register_plugin_manager("transcription", MockPluginManager())
    registry.register_plugin_manager("llm", MockLLMManager(), display_name="Language Models")
    
    # Show all categories and plugins
    print("Categories:")
    for category in registry.get_categories():
        display = registry.get_category_display_name(category)
        plugins = registry.get_plugins_by_category(category)
        print(f"  {display} ({category}): {len(plugins)} plugins")
    
    # Show all plugins
    print(f"\nTotal plugins: {len(registry.get_all_plugins())}")
    for plugin in registry.get_all_plugins():
        print(f"  - [{plugin.category}] {plugin.name}")

Categories:
  Language Models (llm): 2 plugins
  Transcription (transcription): 2 plugins

Total plugins: 4
  - [transcription] whisper_tiny
  - [transcription] whisper_base
  - [llm] llama3
  - [llm] gpt4


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()